# 1. Prática

## Extraindo os dados

In [ ]:
!gzip -d NASA_access_log_Aug95.gz >/dev/null 
!gzip -d NASA_access_log_Jul95.gz >/dev/null
print('OK!')

## Copiando os dados para o HDFS
Caminho para os arquivos de entrada: /user/jovyan/input

In [ ]:
!hdfs dfs -mkdir /user/jovyan/input
!hdfs dfs -put NASA_access_log_Aug95 /user/jovyan/input
!hdfs dfs -put NASA_access_log_Jul95 /user/jovyan/input
print('OK!')

## Executando o Script Spark - Nasa log analysis
### Informações obtidas:
#### 1. Número de hosts únicos.
#### 2. O total de erros 404.
#### 3. As 5 URLs que mais causaram erro 404.
#### 4. Quantidade de erros 404 por dia.
#### 5. O total de bytes retornados.

In [ ]:
!spark-submit --master yarn nasa_requests.py

# 2. Teoria

## Qual o objetivo do comando cache em Spark?
O comando cache armazena o RDD (Resilient Distributed Dataset) em memória. Utilizamos a função cache quando queremos reutilizar o RDD. O RDD é reprocessado cada vez que executamos uma ação, utilizando a função cache este RDD ficará disponível na memória aumentando o desempenho de acesso em seu reuso.

## O mesmo código implementado em Spark é normalmente mais rápido que a implementação equivalente em MapReduce. Por quê?
O Spark utiliza o conceito de DAG (Directed Acyclic Graph - Grafos Acíclicos Direcionados) que permite mapear os passos que serão necessários para atingir o resultado esperado e, então, otimiza esses passos da melhor maneira possível; O Spark faz suas operações em memória; O Spark trabalha com RDD que armazena os elementos, é tolerante a falhas e pode ser executado em paralelo. Além disso, o Spark não computa os seus resultados de imediato, uma transformação é executada apenas quando uma ação é solicitada, isto também contribui para o seu desempenho. Por estes motivos, o Spark consegue executar um trabalho de maneira mais eficiente que o MapReduce.

## Qual é a função do SparkContext?
O SparkContext permite a conexão com um cluster Spark e então podemos utilizar os recursos do Spark naquele cluster, por exemplo criar RDDs.

## Explique com suas palavras o que é Resilient Distributed Datasets (RDD).
O RDD é a principal estrutura de dados do Spark, é um conjunto de registros imutáveis que podem ser executados em paralelo. É possível realizar transformações e ações no RDD. Apenas as transformações geram um novo RDD e essas transformações só são aplicadas quando uma ação é necessária. Cada vez que é necessário utilizar um RDD ele é reprocessado, mas existem funções para persistir um RDD em memória ou disco melhorando o desempenho.

## GroupByKey é menos eficiente que reduceByKey em grandes dataset. Por quê?
O GroupByKey é menos eficiente que o reduceByKey porque ele transfere mais dados pela rede para computar o resultado final. O GroupByKey percorre todas as partições em busca de todos os valores para todas as chaves e então transfere todos esses dados via rede para obter o resultado final (processo conhecido como shuffle). Por outro lado, o reduceByKey primeiro agrega todas os valores das chaves, transferindo via rede uma menor quantidade de dados, ou seja, apenas um valor para cada chave.

## Explique o que o código Scala abaixo faz.
---
``` scala
val​​ ​ textFile​​ ​ = ​ ​ sc​ . ​ textFile​ ( ​ "hdfs://..."​ )
val​​ ​ counts​​ ​ = ​ ​ textFile​ . ​ flatMap​ ( ​ line​​ ​ =>​​ ​ line​ . ​ split​ ( ​ " ​ ​ " ​ ))
​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ . ​ map​ ( ​ word​​ ​ =>​​ ​ ( ​ word​ , ​ ​ 1 ​ ))
​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ ​ . ​ reduceByKey​ ( ​ _ ​ ​ + ​ ​ _ ​ )
counts​ . ​ saveAsTextFile​ ( ​ "hdfs://..."​ )
``` 
---

O código realiza a contagem de palavras de um arquivo lido do HDFS e salva seu resultado em um arquivo texto também no HDFS. É criado um RDD a partir do arquivo lido do HDFS, em seguida ocorrem várias transformações: o RDD inicial é transformado utilizando a função flatMap, assim obtemos as palavras separando cada entrada por espaço. Em seguida, transformamos o RDD utilizando a função map para exportar pares chave-valor com a palavras sendo a chave e o valor o numeral 1. Por fim, a transformação reduceBykey irá somar os valores para cada chave. A ação saveAsTextFile faz com que as transformações sejam de fato processadas e salva o resultado em um arquivo no HDFS.
